In [80]:
!pip install contractions


In [83]:
import pandas as pd
import numpy as np
import re
import string
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
import re
import nltk
import contractions
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import contractions
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


# 2. Apply oversampling only on the training set
from imblearn.over_sampling import RandomOverSampler



In [88]:
import nltk
nltk.download('punkt')       # Sentence tokenizer
nltk.download('wordnet')     # Lemmatizer data
nltk.download('omw-1.4')     # WordNet multilingual support
nltk.download('stopwords')   # Stopword list
# %%
import nltk
nltk.download('punkt')       # Sentence tokenizer
nltk.download('wordnet')     # Lemmatizer data
nltk.download('omw-1.4')     # WordNet multilingual support
nltk.download('stopwords')   # Stopword list
nltk.download('punkt_tab')   # Punkt tokenizer tab resource


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Un

True

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
# Initialize
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
# 🧹 Cleaning function
def clean_text(text):
    text = text.lower()
    text = contractions.fix(text)  # expand contractions
    text = re.sub('<.*?>', '', text)  # remove HTML
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove punctuation/numbers
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]  # remove stopwords
    words = [lemmatizer.lemmatize(word) for word in words]  # lemmatize
    return ' '.join(words)


In [92]:

# 1. Load data (adjust path)
df = pd.read_csv('/content/drive/MyDrive/colab,zip/IMDB Dataset.csv')[:8000]  # you can choose your size


In [93]:
# Assuming your dataframe is df with columns 'review' and 'sentiment'
df['cleaned_review'] = df['review'].apply(clean_text)

# Label encode sentiments
le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'])  # e.g. positive=1, negative=0

X = df['cleaned_review']
y = df['label']


In [94]:
# 5. Split the data (stratify to keep class balance in splits)
x_train, x_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=101, stratify=y
)



In [95]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

# Pad sequences to the same length
max_length = 100  # or choose based on data analysis
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')


In [ ]:
# After tokenizing and padding your train and test data:
ros = RandomOverSampler(random_state=42)
x_train_oversampled, y_train_oversampled = ros.fit_resample(train_padded, y_train)

model = Sequential([
    Embedding(input_dim=max_words, output_dim=64),
    Bidirectional(GRU(64, dropout=0.4, recurrent_dropout=0.2)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    x_train_oversampled, y_train_oversampled,
    epochs=10,
    batch_size=32,
    validation_data=(test_padded, y_test),
    callbacks=[early_stop]
)


Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 63s 264ms/step - accuracy: 0.5314 - loss: 0.7261 - val_accuracy: 0.7337 - val_loss: 0.6566
Epoch 2/10
  3/201 ━━━━━━━━━━━━━━━━━━━━ 40s 206ms/step - accuracy: 0.8368 - loss: 0.3763

In [ ]:
# 10. Save model and tokenizer for deployment
model.save("gru_sentiment_model.keras")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)
